# A Vision Transformer without Attention

## imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_addons as tfa

from emgdecompy.decomposition import *
from emgdecompy.contrast import *
from emgdecompy.viz import *
from emgdecompy.preprocessing import *
from db1_preprocess_utils import *
from feature_extraction import *
from experiment_one_utils import *

from tensorflow.keras import layers
from tensorflow import keras
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import tensorflow as tf
import numpy as np
from sklearn.model_selection import KFold
from scipy.signal import stft
from tensorflow.keras import layers


# Setting seed for reproducibiltiy
SEED = 42
keras.utils.set_random_seed(SEED)

## Hyperparameters

In [2]:
class Config(object):
    # DATA
    batch_size = 36 
    buffer_size = batch_size * 2
    num_classes = 34
    

    # ARCHITECTURE
    patch_size = 4
    projected_dim = 96
    num_shift_blocks_per_stages = [2, 4, 8, 2]
    epsilon = 1e-5
    stochastic_depth_rate = 0.2
    mlp_dropout_rate = 0.2
    num_div = 12
    shift_pixel = 1
    mlp_expand_ratio = 2

    # OPTIMIZER
    lr_start = 1e-5
    lr_max = 1e-3
    weight_decay = 1e-4

    # TRAINING
    epochs = 150


config = Config()

## Import data

In [3]:
path = 'D:\\AI-Workspace\\sEMGClassification\\GestureClassificationUsingCViT\\data\\DB1\\raw'

signal_type='raw'
input_type='raw'
n_channels=64
low_cut=20
high_cut=50
order=6 
window_length=50
overlap=50
fs=2048

whiten=False
center=True
extend=True
extend_size=1
normalize=False
mu=0

n_folds = 3

start_subject = 15
total_subject = 20
session=[1,2]
ratio = 0.2

noise_db = [5,10,15]
std = 1
type_of_experiment = 3

X_train, y_label, X_test, y_test = get_experiment_data(path, subjects=start_subject, sessions=session,
                                                       signal_type='raw', input_type=input_type, 
                                                       channels=n_channels, low_cut=20, high_cut=500,
                                                      order=6, window_size=window_length, overlap=overlap, fs=fs, 
                                                      extend=extend, center=center, 
                                                      extend_size=extend_size, whiten=whiten,
                                                      normalize=False, mu=0, ratio=ratio)

input_size = X_train.shape[0:]
print(f"Training samples: {len(X_train)}")
print(f"Validation samples: {len(X_test)}")

####################################################################################
Loading subject 15
####################################################################################
Centred.
Extended.
Size of the input data is (17021, 128, 102, 1)
The input label shape is (17021,)
The total number of classes is 34
************************************************************************************
Loaded RAW input data
************************************************************************************
Training samples: 17021
Validation samples: 4256


#### The MLP block

In [4]:
class MLP(layers.Layer):
    """Get the MLP layer for each shift block.

    Args:
        mlp_expand_ratio (int): The ratio with which the first feature map is expanded.
        mlp_dropout_rate (float): The rate for dropout.
    """

    def __init__(self, mlp_expand_ratio, mlp_dropout_rate, **kwargs):
        super().__init__(**kwargs)
        self.mlp_expand_ratio = mlp_expand_ratio
        self.mlp_dropout_rate = mlp_dropout_rate

    def build(self, input_shape):
        input_channels = input_shape[-1]
        initial_filters = int(self.mlp_expand_ratio * input_channels)

        self.mlp = keras.Sequential(
            [
                layers.Dense(units=initial_filters, activation=tf.nn.gelu,),
                layers.Dropout(rate=self.mlp_dropout_rate),
                layers.Dense(units=input_channels),
                layers.Dropout(rate=self.mlp_dropout_rate),
            ]
        )

    def call(self, x):
        x = self.mlp(x)
        return x


#### The DropPath layer

In [5]:

class DropPath(layers.Layer):
    """Drop Path also known as the Stochastic Depth layer.

    Refernece:
        - https://keras.io/examples/vision/cct/#stochastic-depth-for-regularization
        - github.com:rwightman/pytorch-image-models
    """

    def __init__(self, drop_path_prob, **kwargs):
        super().__init__(**kwargs)
        self.drop_path_prob = drop_path_prob

    def call(self, x, training=False):
        if training:
            keep_prob = 1 - self.drop_path_prob
            shape = (tf.shape(x)[0],) + (1,) * (len(tf.shape(x)) - 1)
            random_tensor = keep_prob + tf.random.uniform(shape, 0, 1)
            random_tensor = tf.floor(random_tensor)
            return (x / keep_prob) * random_tensor
        return x


In [6]:
class ShiftViTBlock(layers.Layer):
    """A unit ShiftViT Block

    Args:
        shift_pixel (int): The number of pixels to shift. Default to 1.
        mlp_expand_ratio (int): The ratio with which MLP features are
            expanded. Default to 2.
        mlp_dropout_rate (float): The dropout rate used in MLP.
        num_div (int): The number of divisions of the feature map's channel.
            Totally, 4/num_div of channels will be shifted. Defaults to 12.
        epsilon (float): Epsilon constant.
        drop_path_prob (float): The drop probability for drop path.
    """

    def __init__(
        self,
        epsilon,
        drop_path_prob,
        mlp_dropout_rate,
        num_div=12,
        shift_pixel=1,
        mlp_expand_ratio=2,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.shift_pixel = shift_pixel
        self.mlp_expand_ratio = mlp_expand_ratio
        self.mlp_dropout_rate = mlp_dropout_rate
        self.num_div = num_div
        self.epsilon = epsilon
        self.drop_path_prob = drop_path_prob

    def build(self, input_shape):
        self.H = input_shape[1]
        self.W = input_shape[2]
        self.C = input_shape[3]
        self.layer_norm = layers.LayerNormalization(epsilon=self.epsilon)
        self.drop_path = (
            DropPath(drop_path_prob=self.drop_path_prob)
            if self.drop_path_prob > 0.0
            else layers.Activation("linear")
        )
        self.mlp = MLP(
            mlp_expand_ratio=self.mlp_expand_ratio,
            mlp_dropout_rate=self.mlp_dropout_rate,
        )

    def get_shift_pad(self, x, mode):
        """Shifts the channels according to the mode chosen."""
        if mode == "left":
            offset_height = 0
            offset_width = 0
            target_height = 0
            target_width = self.shift_pixel
        elif mode == "right":
            offset_height = 0
            offset_width = self.shift_pixel
            target_height = 0
            target_width = self.shift_pixel
        elif mode == "up":
            offset_height = 0
            offset_width = 0
            target_height = self.shift_pixel
            target_width = 0
        else:
            offset_height = self.shift_pixel
            offset_width = 0
            target_height = self.shift_pixel
            target_width = 0
        crop = tf.image.crop_to_bounding_box(
            x,
            offset_height=offset_height,
            offset_width=offset_width,
            target_height=self.H - target_height,
            target_width=self.W - target_width,
        )
        shift_pad = tf.image.pad_to_bounding_box(
            crop,
            offset_height=offset_height,
            offset_width=offset_width,
            target_height=self.H,
            target_width=self.W,
        )
        return shift_pad

    def call(self, x, training=False):
        # Split the feature maps
        x_splits = tf.split(x, num_or_size_splits=self.C // self.num_div, axis=-1)

        # Shift the feature maps
        x_splits[0] = self.get_shift_pad(x_splits[0], mode="left")
        x_splits[1] = self.get_shift_pad(x_splits[1], mode="right")
        x_splits[2] = self.get_shift_pad(x_splits[2], mode="up")
        x_splits[3] = self.get_shift_pad(x_splits[3], mode="down")

        # Concatenate the shifted and unshifted feature maps
        x = tf.concat(x_splits, axis=-1)

        # Add the residual connection
        shortcut = x
        x = shortcut + self.drop_path(self.mlp(self.layer_norm(x)), training=training)
        return x


#### The PatchMerging layer

In [7]:

class PatchMerging(layers.Layer):
    """The Patch Merging layer.

    Args:
        epsilon (float): The epsilon constant.
    """

    def __init__(self, epsilon, **kwargs):
        super().__init__(**kwargs)
        self.epsilon = epsilon

    def build(self, input_shape):
        filters = 2 * input_shape[-1]
        self.reduction = layers.Conv2D(
            filters=filters, kernel_size=2, strides=2, padding="same", use_bias=False
        )
        self.layer_norm = layers.LayerNormalization(epsilon=self.epsilon)

    def call(self, x):
        # Apply the patch merging algorithm on the feature maps
        x = self.layer_norm(x)
        x = self.reduction(x)
        return x


In [8]:
input_size

(17021, 128, 102, 1)

#### Stacked Shift Blocks

In [9]:
# Note: This layer will have a different depth of stacking
# for different stages on the model.
class StackedShiftBlocks(layers.Layer):
    """The layer containing stacked ShiftViTBlocks.

    Args:
        epsilon (float): The epsilon constant.
        mlp_dropout_rate (float): The dropout rate used in the MLP block.
        num_shift_blocks (int): The number of shift vit blocks for this stage.
        stochastic_depth_rate (float): The maximum drop path rate chosen.
        is_merge (boolean): A flag that determines the use of the Patch Merge
            layer after the shift vit blocks.
        num_div (int): The division of channels of the feature map. Defaults to 12.
        shift_pixel (int): The number of pixels to shift. Defaults to 1.
        mlp_expand_ratio (int): The ratio with which the initial dense layer of
            the MLP is expanded Defaults to 2.
    """

    def __init__(
        self,
        epsilon,
        mlp_dropout_rate,
        num_shift_blocks,
        stochastic_depth_rate,
        is_merge,
        num_div=12,
        shift_pixel=1,
        mlp_expand_ratio=2,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.epsilon = epsilon
        self.mlp_dropout_rate = mlp_dropout_rate
        self.num_shift_blocks = num_shift_blocks
        self.stochastic_depth_rate = stochastic_depth_rate
        self.is_merge = is_merge
        self.num_div = num_div
        self.shift_pixel = shift_pixel
        self.mlp_expand_ratio = mlp_expand_ratio

    def build(self, input_shapes):
        # Calculate stochastic depth probabilities.
        # Reference: https://keras.io/examples/vision/cct/#the-final-cct-model
        dpr = [
            x
            for x in np.linspace(
                start=0, stop=self.stochastic_depth_rate, num=self.num_shift_blocks
            )
        ]

        # Build the shift blocks as a list of ShiftViT Blocks
        self.shift_blocks = list()
        for num in range(self.num_shift_blocks):
            self.shift_blocks.append(
                ShiftViTBlock(
                    num_div=self.num_div,
                    epsilon=self.epsilon,
                    drop_path_prob=dpr[num],
                    mlp_dropout_rate=self.mlp_dropout_rate,
                    shift_pixel=self.shift_pixel,
                    mlp_expand_ratio=self.mlp_expand_ratio,
                )
            )
        if self.is_merge:
            self.patch_merge = PatchMerging(epsilon=self.epsilon)

    def call(self, x, training=False):
        for shift_block in self.shift_blocks:
            x = shift_block(x, training=training)
        if self.is_merge:
            x = self.patch_merge(x)
        return x


## The ShiftViT model

In [10]:
class ShiftViTModel(keras.Model):
    """The ShiftViT Model.

    Args:
        data_augmentation (keras.Model): A data augmentation model.
        projected_dim (int): The dimension to which the patches of the image are
            projected.
        patch_size (int): The patch size of the images.
        num_shift_blocks_per_stages (list[int]): A list of all the number of shit
            blocks per stage.
        epsilon (float): The epsilon constant.
        mlp_dropout_rate (float): The dropout rate used in the MLP block.
        stochastic_depth_rate (float): The maximum drop rate probability.
        num_div (int): The number of divisions of the channesl of the feature
            map. Defaults to 12.
        shift_pixel (int): The number of pixel to shift. Default to 1.
        mlp_expand_ratio (int): The ratio with which the initial mlp dense layer
            is expanded to. Defaults to 2.
    """

    def __init__(
        self,
        projected_dim,
        patch_size,
        num_shift_blocks_per_stages,
        epsilon,
        mlp_dropout_rate,
        stochastic_depth_rate,
        num_div=12,
        shift_pixel=1,
        mlp_expand_ratio=2,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.patch_projection = layers.Conv2D(
            filters=projected_dim,
            kernel_size=patch_size,
            strides=patch_size,
            padding="same",
        )
        self.stages = list()
        for index, num_shift_blocks in enumerate(num_shift_blocks_per_stages):
            if index == len(num_shift_blocks_per_stages) - 1:
                # This is the last stage, do not use the patch merge here.
                is_merge = False
            else:
                is_merge = True
            # Build the stages.
            self.stages.append(
                StackedShiftBlocks(
                    epsilon=epsilon,
                    mlp_dropout_rate=mlp_dropout_rate,
                    num_shift_blocks=num_shift_blocks,
                    stochastic_depth_rate=stochastic_depth_rate,
                    is_merge=is_merge,
                    num_div=num_div,
                    shift_pixel=shift_pixel,
                    mlp_expand_ratio=mlp_expand_ratio,
                )
            )
        self.global_avg_pool = layers.GlobalAveragePooling2D()

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "patch_projection": self.patch_projection,
                "stages": self.stages,
                "global_avg_pool": self.global_avg_pool,
            }
        )
        return config

    def _calculate_loss(self, data, training=False):
        (images, labels) = data

        # Create patches and project the pathces.
        projected_patches = self.patch_projection(images)

        # Pass through the stages
        x = projected_patches
        for stage in self.stages:
            x = stage(x, training=training)

        # Get the logits.
        logits = self.global_avg_pool(x)

        # Calculate the loss and return it.
        total_loss = self.compiled_loss(labels, logits)
        return total_loss, labels, logits

    def train_step(self, inputs):
        with tf.GradientTape() as tape:
            total_loss, labels, logits = self._calculate_loss(
                data=inputs, training=True
            )

        # Apply gradients.
        train_vars = [
            self.patch_projection.trainable_variables,
            self.global_avg_pool.trainable_variables,
        ]
        train_vars = train_vars + [stage.trainable_variables for stage in self.stages]

        # Optimize the gradients.
        grads = tape.gradient(total_loss, train_vars)
        trainable_variable_list = []
        for (grad, var) in zip(grads, train_vars):
            for g, v in zip(grad, var):
                trainable_variable_list.append((g, v))
        self.optimizer.apply_gradients(trainable_variable_list)

        # Update the metrics
        self.compiled_metrics.update_state(labels, logits)
        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        _, labels, logits = self._calculate_loss(data=data, training=False)

        # Update the metrics
        self.compiled_metrics.update_state(labels, logits)
        return {m.name: m.result() for m in self.metrics}


## Instantiate the model

In [11]:
model = ShiftViTModel(
    projected_dim=config.projected_dim,
    patch_size=config.patch_size,
    num_shift_blocks_per_stages=config.num_shift_blocks_per_stages,
    epsilon=config.epsilon,
    mlp_dropout_rate=config.mlp_dropout_rate,
    stochastic_depth_rate=config.stochastic_depth_rate,
    num_div=config.num_div,
    shift_pixel=config.shift_pixel,
    mlp_expand_ratio=config.mlp_expand_ratio,
)

## Learning rate schedule

In [12]:
# Some code is taken from:
# https://www.kaggle.com/ashusma/training-rfcx-tensorflow-tpu-effnet-b2.
class WarmUpCosine(keras.optimizers.schedules.LearningRateSchedule):
    """A LearningRateSchedule that uses a warmup cosine decay schedule."""

    def __init__(self, lr_start, lr_max, warmup_steps, total_steps):
        """
        Args:
            lr_start: The initial learning rate
            lr_max: The maximum learning rate to which lr should increase to in
                the warmup steps
            warmup_steps: The number of steps for which the model warms up
            total_steps: The total number of steps for the model training
        """
        super().__init__()
        self.lr_start = lr_start
        self.lr_max = lr_max
        self.warmup_steps = warmup_steps
        self.total_steps = total_steps
        self.pi = tf.constant(np.pi)

    def __call__(self, step):
        # Check whether the total number of steps is larger than the warmup
        # steps. If not, then throw a value error.
        if self.total_steps < self.warmup_steps:
            raise ValueError(
                f"Total number of steps {self.total_steps} must be"
                + f"larger or equal to warmup steps {self.warmup_steps}."
            )

        # `cos_annealed_lr` is a graph that increases to 1 from the initial
        # step to the warmup step. After that this graph decays to -1 at the
        # final step mark.
        cos_annealed_lr = tf.cos(
            self.pi
            * (tf.cast(step, tf.float32) - self.warmup_steps)
            / tf.cast(self.total_steps - self.warmup_steps, tf.float32)
        )

        # Shift the mean of the `cos_annealed_lr` graph to 1. Now the grpah goes
        # from 0 to 2. Normalize the graph with 0.5 so that now it goes from 0
        # to 1. With the normalized graph we scale it with `lr_max` such that
        # it goes from 0 to `lr_max`
        learning_rate = 0.5 * self.lr_max * (1 + cos_annealed_lr)

        # Check whether warmup_steps is more than 0.
        if self.warmup_steps > 0:
            # Check whether lr_max is larger that lr_start. If not, throw a value
            # error.
            if self.lr_max < self.lr_start:
                raise ValueError(
                    f"lr_start {self.lr_start} must be smaller or"
                    + f"equal to lr_max {self.lr_max}."
                )

            # Calculate the slope with which the learning rate should increase
            # in the warumup schedule. The formula for slope is m = ((b-a)/steps)
            slope = (self.lr_max - self.lr_start) / self.warmup_steps

            # With the formula for a straight line (y = mx+c) build the warmup
            # schedule
            warmup_rate = slope * tf.cast(step, tf.float32) + self.lr_start

            # When the current step is lesser that warmup steps, get the line
            # graph. When the current step is greater than the warmup steps, get
            # the scaled cos graph.
            learning_rate = tf.where(
                step < self.warmup_steps, warmup_rate, learning_rate
            )

        # When the current step is more that the total steps, return 0 else return
        # the calculated graph.
        return tf.where(
            step > self.total_steps, 0.0, learning_rate, name="learning_rate"
        )

## Run Experiment

In [13]:
def run_experiment(input_size, n_folds, n_batches, n_epochs, start_subject, total_subject, session, path, 
                       input_type,n_channels, extend, extend_size, center, whiten, ratio, noise_db, std,
                       dims, patch_size, num_shift_blocks_per_stages,
                        epsilon, mlp_dropout_rate, stochastic_depth_rate, num_div, shift_pixel,
                        mlp_expand_ratio, type_of_experiment, lr_start, lr_max, weight_decay):
   
    
    if  type_of_experiment == 1:
        print('************************************************')
        print('Experiment One')
        print('************************************************')
    elif type_of_experiment == 2:
        print('************************************************')
        print('Experiment Two')
        print('************************************************')
    elif type_of_experiment == 3:
        print('************************************************')
        print('Experiment Three')
        print('************************************************')
    else:
        print('************************************************')
        print('Enter Valid Experiment')
        print('************************************************')
        
        
    result = pd.DataFrame({
    'Subject': [0],
    'Validation_accuracy': [0],
    'No_noise': [0],
    '5_dB': [0],
    '10_dB': [0],
    '15_dB': [0],
    'Fold_1': [0],
    'Fold_2': [0],
    'Fold_3': [0]
    })
    
            
    total_steps = int((int(input_size[0]) / n_batches) * n_epochs)

    # Calculate the number of steps for warmup.
    warmup_epoch_percentage = 0.15
    warmup_steps = int(total_steps * warmup_epoch_percentage)

    # Initialize the warmupcosine schedule.
    scheduled_lrs = WarmUpCosine(lr_start=lr_start, lr_max=lr_max, warmup_steps=warmup_steps, total_steps=total_steps)
    
    
    for s in range(start_subject, (total_subject+1)):
        
        X_train, y_train, X_test, y_test = get_experiment_data(path, subjects=s, sessions=session,
                                                       signal_type='raw', input_type=input_type, 
                                                       channels=n_channels, low_cut=20, high_cut=500,
                                                      order=6, window_size=50, overlap=50, fs=2048, 
                                                      extend=extend,  extend_size=extend_size, 
                                                      center=center, whiten=whiten,
                                                      normalize=False, mu=0, ratio=ratio)

    
        #X_train, y_train, X_test, y_test = spilt_data(data, label, ratio)
        if input_type == 'raw':
            print('Adding noise to RAW input test data')
            X_test = np.expand_dims(X_test, axis=3)
            X_test_1 = add_noise_all_channel(X_test, noise_db[0], std)
            X_test_2 = add_noise_all_channel(X_test, noise_db[1], std)
            X_test_3 = add_noise_all_channel(X_test, noise_db[2], std)
            
            print("Size of the input test data is {}".format(X_test_3.shape))
      
            
        elif input_type == 'tkeo':
            print('Adding noise to  TKEO input test data')
            X_test = tkeo_image(X_test)
            X_test = np.expand_dims(X_test, axis=3)
            
            X_test_1 = add_noise_all_channel(X_test, noise_db[0], std)
            X_test_2 = add_noise_all_channel(X_test, noise_db[1], std)
            X_test_3 = add_noise_all_channel(X_test, noise_db[2], std)
            
            
            print("Size of the input test data is {}".format(X_test_3.shape))

            
        elif input_type == 'stft':
            print('Adding noise to STFT input test data')
            
            
            X_test_1 = add_noise_all_channel(X_test, noise_db[0], std)
            X_test_1 = stft_image(X_test_1, samples=X_test_1.shape[2])
            
            X_test_2 = add_noise_all_channel(X_test, noise_db[1], std)
            X_test_2 = stft_image(X_test_2, samples=X_test_2.shape[2])
            
            X_test_3 = add_noise_all_channel(X_test, noise_db[2], std)
            X_test_3 = stft_image(X_test_3, samples=X_test_3.shape[2])
            
            X_test = stft_image(X_test, samples=X_test.shape[2])
            
            
            print("Size of the input test data is {}".format(X_test_3.shape))
            
        else:
            print('Use correct input type')
        
        
        
        # Get the optimizer.
        optimizer = tfa.optimizers.AdamW(learning_rate=scheduled_lrs, weight_decay=weight_decay)
        
        kfold = KFold(n_splits=n_folds, shuffle=False)
        accuracy_per_fold = []
        loss_per_fold = []
        fold_no = 1
        
        convergence_speed = []
        
        callback = tf.keras.callbacks.EarlyStopping(monitor="accuracy", patience=10, mode="auto")
        
        for train, test in kfold.split(X_train, y_train):
        
            
            
            model = ShiftViTModel(
            projected_dim=dims,
            patch_size=patch_size,
            num_shift_blocks_per_stages=num_shift_blocks_per_stages,
            epsilon=epsilon,
            mlp_dropout_rate=mlp_dropout_rate,
            stochastic_depth_rate=stochastic_depth_rate,
            num_div=num_div,
            shift_pixel=shift_pixel,
            mlp_expand_ratio=mlp_expand_ratio,
            )
            
            
            
            
            model.compile(optimizer=optimizer,
                          loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                          metrics=[keras.metrics.SparseCategoricalAccuracy(name="accuracy")])
            
            print('---------------------------------------------------')
            print(f'Training for fold {fold_no} -------')
            
            history = model.fit(X_train[train], y_train[train], batch_size=n_batches, 
                                epochs= n_epochs, verbose=1, 
                                callbacks=callback)
            
            conv_speed = len(history.history['accuracy'])
            convergence_speed.append(conv_speed)
            
            scores = model.evaluate(X_train[test], y_train[test], verbose=0)
            print(f'Score for fold  {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
            accuracy_per_fold.append(scores[1] *100)
            loss_per_fold.append(scores[0])
                  
            fold_no = fold_no + 1
        
        print("Average Score per fold ")
    
        for i in range(0, len(accuracy_per_fold)):
            print('-----------------------------------------------')
            print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {accuracy_per_fold[i]}%')
        print('-----------------------------------------------')
        print('Average Metrics for all folds: ')
        print(f'> Accuracy: {np.mean(accuracy_per_fold)} (+- {np.std(accuracy_per_fold)})')
        print(f'> Loss: {np.mean(loss_per_fold)}')
        print('-----------------------------------------------')
        
        print('************************************************')
        print(f'For subject {s} without noise')
        scores_0 = model.evaluate(X_test, y_test, verbose=0)
        print('The loss is {} and accuracy is {}%'.format(scores_0[0], (scores_0[1]*100)))
        print(f'For subject {s} and noise 5 dB')
        scores_1 = model.evaluate(X_test_1, y_test, verbose=0)
        print('The loss is {} and accuracy is {}%'.format(scores_1[0], (scores_1[1]*100)))
        print(f'For subject {s} and noise 10 dB')
        scores_2 = model.evaluate(X_test_2, y_test, verbose=0)
        print('The loss is {} and accuracy is {}%'.format(scores_2[0], (scores_2[1]*100)))
        print(f'For subject {s} and noise 15 dB')  
        scores_3 = model.evaluate(X_test_3, y_test, verbose=0) 
        print('The loss is {} and accuracy is {}%'.format(scores_3[0], (scores_3[1]*100)))
        print('************************************************') 

        
        result.at[s-1, 'Subject'] =  s
        result.at[s-1, 'Validation_accuracy'] =  np.mean(accuracy_per_fold)       
        result.at[s-1, 'No_noise'] =  scores_0[1]*100             
        result.at[s-1, '5_dB'] =  scores_1[1]*100    
        result.at[s-1, '10_dB'] =  scores_2[1]*100
        result.at[s-1, '15_dB'] =  scores_3[1]*100   
        
        result.at[s-1, 'Fold_1'] =  convergence_speed[0]
        result.at[s-1, 'Fold_2'] =  convergence_speed[1]
        result.at[s-1, 'Fold_3'] =  convergence_speed[2] 
        
        save_path = 'without_attention_raw_all.csv'
        result.to_csv(save_path, index=False)

In [14]:
run_experiment(input_size, n_folds,  config.batch_size, config.epochs, start_subject, total_subject, session, path, 
                       input_type,n_channels, extend, extend_size, center, whiten, ratio, noise_db, std,
                       config.projected_dim, config.patch_size, config.num_shift_blocks_per_stages,
                        config.epsilon, config.mlp_dropout_rate, config.stochastic_depth_rate, config.num_div, config.shift_pixel,
                        config.mlp_expand_ratio, type_of_experiment, config.lr_start, config.lr_max, config.weight_decay)

************************************************
Experiment Three
************************************************
####################################################################################
Loading subject 15
####################################################################################
Centred.
Extended.
Size of the input data is (17021, 128, 102, 1)
The input label shape is (17021,)
The total number of classes is 34
************************************************************************************
Loaded RAW input data
************************************************************************************
Adding noise to RAW input test data
Size of the input test data is (4256, 128, 102, 1)
---------------------------------------------------
Training for fold 1 -------
Epoch 1/150
316/316 [==============================] - 71s 152ms/step - loss: 4.2251 - accuracy: 0.0276
Epoch 2/150
316/316 [==============================] - 48s 152ms/step - loss: 3.3520 - accuracy: 0.

316/316 [==============================] - 52s 163ms/step - loss: 0.0546 - accuracy: 0.9841
Epoch 72/150
316/316 [==============================] - 49s 156ms/step - loss: 0.0307 - accuracy: 0.9907
Epoch 73/150
316/316 [==============================] - 49s 154ms/step - loss: 0.0243 - accuracy: 0.9932
Epoch 74/150
316/316 [==============================] - 49s 156ms/step - loss: 0.0249 - accuracy: 0.9935
Score for fold  1: loss of 0.09781122952699661; accuracy of 97.25061655044556%
---------------------------------------------------
Training for fold 2 -------
Epoch 1/150
316/316 [==============================] - 72s 159ms/step - loss: 4.7021 - accuracy: 0.0311
Epoch 2/150
316/316 [==============================] - 51s 160ms/step - loss: 3.6357 - accuracy: 0.0292
Epoch 3/150
316/316 [==============================] - 50s 158ms/step - loss: 3.5913 - accuracy: 0.0330
Epoch 4/150
316/316 [==============================] - 50s 158ms/step - loss: 3.5808 - accuracy: 0.0265
Epoch 5/150
316/31

316/316 [==============================] - 48s 150ms/step - loss: 0.0300 - accuracy: 0.9944
Epoch 74/150
316/316 [==============================] - 48s 152ms/step - loss: 0.0343 - accuracy: 0.9919
Epoch 75/150
316/316 [==============================] - 48s 151ms/step - loss: 0.0549 - accuracy: 0.9884
Score for fold  2: loss of 0.05890681594610214; accuracy of 98.06133508682251%
---------------------------------------------------
Training for fold 3 -------
Epoch 1/150
316/316 [==============================] - 67s 150ms/step - loss: 3.9020 - accuracy: 0.0444
Epoch 2/150
316/316 [==============================] - 48s 152ms/step - loss: 3.1542 - accuracy: 0.0967
Epoch 3/150
316/316 [==============================] - 48s 151ms/step - loss: 2.5156 - accuracy: 0.2528
Epoch 4/150
316/316 [==============================] - 48s 152ms/step - loss: 1.7488 - accuracy: 0.4743
Epoch 5/150
316/316 [==============================] - 48s 150ms/step - loss: 1.1675 - accuracy: 0.6624
Epoch 6/150
316/316

313/313 [==============================] - 47s 151ms/step - loss: 0.7397 - accuracy: 0.7933
Epoch 9/150
313/313 [==============================] - 48s 152ms/step - loss: 0.6213 - accuracy: 0.8264
Epoch 10/150
313/313 [==============================] - 47s 151ms/step - loss: 0.5291 - accuracy: 0.8493
Epoch 11/150
313/313 [==============================] - 47s 150ms/step - loss: 0.4680 - accuracy: 0.8651
Epoch 12/150
313/313 [==============================] - 47s 151ms/step - loss: 0.4007 - accuracy: 0.8836
Epoch 13/150
313/313 [==============================] - 48s 152ms/step - loss: 0.3740 - accuracy: 0.8912
Epoch 14/150
313/313 [==============================] - 47s 152ms/step - loss: 0.3533 - accuracy: 0.8945
Epoch 15/150
313/313 [==============================] - 47s 151ms/step - loss: 0.3057 - accuracy: 0.9135
Epoch 16/150
313/313 [==============================] - 47s 152ms/step - loss: 0.2837 - accuracy: 0.9164
Epoch 17/150
313/313 [==============================] - 47s 151ms/ste

313/313 [==============================] - 48s 152ms/step - loss: 3.5461 - accuracy: 0.0264
Epoch 11/150
313/313 [==============================] - 48s 152ms/step - loss: 3.5399 - accuracy: 0.0312
Epoch 12/150
313/313 [==============================] - 47s 151ms/step - loss: 3.5014 - accuracy: 0.0426
Epoch 13/150
313/313 [==============================] - 47s 152ms/step - loss: 3.3260 - accuracy: 0.0805
Epoch 14/150
313/313 [==============================] - 47s 151ms/step - loss: 2.8099 - accuracy: 0.1819
Epoch 15/150
313/313 [==============================] - 47s 151ms/step - loss: 2.4579 - accuracy: 0.2837
Epoch 16/150
313/313 [==============================] - 47s 152ms/step - loss: 2.1692 - accuracy: 0.3593
Epoch 17/150
313/313 [==============================] - 48s 152ms/step - loss: 1.9741 - accuracy: 0.4198
Epoch 18/150
313/313 [==============================] - 48s 152ms/step - loss: 1.6666 - accuracy: 0.5118
Epoch 19/150
313/313 [==============================] - 47s 151ms/st

...

316/316 [==============================] - 49s 154ms/step - loss: 0.0558 - accuracy: 0.9834
Epoch 93/150
316/316 [==============================] - 49s 154ms/step - loss: 0.0388 - accuracy: 0.9886
Epoch 94/150
316/316 [==============================] - 49s 157ms/step - loss: 0.0480 - accuracy: 0.9858
Epoch 95/150
316/316 [==============================] - 50s 159ms/step - loss: 0.0280 - accuracy: 0.9927
Epoch 96/150
316/316 [==============================] - 50s 159ms/step - loss: 0.0502 - accuracy: 0.9862
Epoch 97/150
316/316 [==============================] - 49s 155ms/step - loss: 0.0388 - accuracy: 0.9905
Epoch 98/150
316/316 [==============================] - 53s 169ms/step - loss: 0.0350 - accuracy: 0.9909
Epoch 99/150
316/316 [==============================] - 51s 163ms/step - loss: 0.0390 - accuracy: 0.9890
Epoch 100/150
316/316 [==============================] - 50s 159ms/step - loss: 0.0465 - accuracy: 0.9861
Epoch 101/150
316/316 [==============================] - 50s 158ms/

316/316 [==============================] - 49s 154ms/step - loss: 0.0839 - accuracy: 0.9805
Epoch 64/150
316/316 [==============================] - 48s 151ms/step - loss: 0.0785 - accuracy: 0.9837
Epoch 65/150
316/316 [==============================] - 47s 150ms/step - loss: 0.1031 - accuracy: 0.9741
Score for fold  2: loss of 0.1340254843235016; accuracy of 95.98166942596436%
---------------------------------------------------
Training for fold 3 -------
Epoch 1/150
316/316 [==============================] - 70s 154ms/step - loss: 3.8002 - accuracy: 0.0457
Epoch 2/150
316/316 [==============================] - 49s 154ms/step - loss: 2.7156 - accuracy: 0.2182
Epoch 3/150
316/316 [==============================] - 48s 152ms/step - loss: 1.6268 - accuracy: 0.5233
Epoch 4/150
316/316 [==============================] - 53s 169ms/step - loss: 1.1813 - accuracy: 0.6637
Epoch 5/150
316/316 [==============================] - 51s 162ms/step - loss: 0.9444 - accuracy: 0.7343
Epoch 6/150
316/316 

323/323 [==============================] - 49s 151ms/step - loss: 0.3323 - accuracy: 0.8994
Epoch 18/150
323/323 [==============================] - 49s 151ms/step - loss: 0.3091 - accuracy: 0.9089
Epoch 19/150
323/323 [==============================] - 49s 152ms/step - loss: 0.3000 - accuracy: 0.9124
Epoch 20/150
323/323 [==============================] - 49s 151ms/step - loss: 0.2650 - accuracy: 0.9203
Epoch 21/150
323/323 [==============================] - 49s 151ms/step - loss: 0.2875 - accuracy: 0.9111
Epoch 22/150
323/323 [==============================] - 49s 151ms/step - loss: 0.2480 - accuracy: 0.9244
Epoch 23/150
323/323 [==============================] - 49s 151ms/step - loss: 0.2257 - accuracy: 0.9299
Epoch 24/150
323/323 [==============================] - 49s 151ms/step - loss: 0.2343 - accuracy: 0.9305
Epoch 25/150
323/323 [==============================] - 49s 151ms/step - loss: 0.2098 - accuracy: 0.9366
Epoch 26/150
323/323 [==============================] - 49s 151ms/st

323/323 [==============================] - 49s 151ms/step - loss: 0.2353 - accuracy: 0.9323
Epoch 22/150
323/323 [==============================] - 49s 151ms/step - loss: 0.2181 - accuracy: 0.9344
Epoch 23/150
323/323 [==============================] - 49s 152ms/step - loss: 0.1961 - accuracy: 0.9412
Epoch 24/150
323/323 [==============================] - 49s 151ms/step - loss: 0.1771 - accuracy: 0.9492
Epoch 25/150
323/323 [==============================] - 49s 151ms/step - loss: 0.1409 - accuracy: 0.9583
Epoch 26/150
323/323 [==============================] - 49s 151ms/step - loss: 0.1403 - accuracy: 0.9584
Epoch 27/150
323/323 [==============================] - 48s 150ms/step - loss: 0.1335 - accuracy: 0.9598
Epoch 28/150
323/323 [==============================] - 49s 151ms/step - loss: 0.1143 - accuracy: 0.9688
Epoch 29/150
323/323 [==============================] - 49s 151ms/step - loss: 0.0975 - accuracy: 0.9716
Epoch 30/150
323/323 [==============================] - 49s 151ms/st

323/323 [==============================] - 49s 150ms/step - loss: 0.0769 - accuracy: 0.9830
Epoch 48/150
323/323 [==============================] - 49s 151ms/step - loss: 0.0592 - accuracy: 0.9909
Epoch 49/150
323/323 [==============================] - 49s 151ms/step - loss: 0.0732 - accuracy: 0.9856
Epoch 50/150
323/323 [==============================] - 49s 151ms/step - loss: 0.0692 - accuracy: 0.9870
Epoch 51/150
323/323 [==============================] - 49s 151ms/step - loss: 0.0743 - accuracy: 0.9867
Epoch 52/150
323/323 [==============================] - 49s 152ms/step - loss: 0.0721 - accuracy: 0.9879
Epoch 53/150
323/323 [==============================] - 49s 151ms/step - loss: 0.0753 - accuracy: 0.9860
Epoch 54/150
323/323 [==============================] - 49s 151ms/step - loss: 0.0825 - accuracy: 0.9836
Epoch 55/150
323/323 [==============================] - 49s 151ms/step - loss: 0.0812 - accuracy: 0.9849
Epoch 56/150
323/323 [==============================] - 49s 151ms/st